# Используем Python для работы с базой данных PostgreSQL

<a target="_blank" href="https://colab.research.google.com/github/sozykin/middle_python/blob/main/05/05_postgresql.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
# Подключаем библиотеку psycopg2
import psycopg2

## Создаем соединение с базой данных

In [2]:
connection = psycopg2.connect(
    database='postgres', # Название базы данных
    user='postgres',     # Имя пользователя
    password='1234qwer', # Пароль (не включайте пароли в продуктивный код!!!)
    host='localhost',    # Имя хоста сервера PostgreSQL
    port=5432,           # Порт, на котором работает сервер PostgreSQL
)

In [3]:
# Создаем курсор
cursor = connection.cursor()

## Создание таблицы для данных

In [5]:
# Оператор SQL для создания таблицы
create_names_table = """
CREATE TABLE IF NOT EXISTS names(
    id INTEGER,
    name TEXT,
    number_of_persons INTEGER,
    global_id INTEGER,
    year INTEGER,
    month TEXT 
)
"""

# Запускаем команду создания таблицы
cursor.execute(create_names_table)

# Фиксируем изменения
connection.commit()

## Вставка данных в таблицу

In [6]:
# Оператор SQL для вставки данных в таблицу 
insert_data = """
INSERT INTO names (id, name, number_of_persons, global_id, year, month)
VALUES (1, 'Мария', 252, 37750254, 2015, 'январь')
"""

# Запускаем команду вставки данных
cursor.execute(insert_data)

# Фиксируем изменения
connection.commit()

## Запрос данных из таблицы

In [7]:
# Запрос SQL для извлечения данных из таблицы names
select_data = "SELECT * FROM names"

# Запускаем запрос получения данных
cursor.execute(select_data)

# Получаем результаты запроса
resuls = cursor.fetchall()

# Печатаем результаты
for row in resuls:
    print(row)

(1, 'Мария', 252, 37750254, 2015, 'январь')


## Завершение работы

In [8]:
# Закрываем курсор
cursor.close()

In [9]:
# Закрываем соединение
connection.close()

## Работа с транзакциями

In [10]:
connection = psycopg2.connect(
    database='postgres', # Название базы данных
    user='postgres',     # Имя пользователя
    password='1234qwer', # Пароль (не включайте пароли в продуктивный код!!!)
    host='localhost',    # Имя хоста сервера PostgreSQL
    port=5432,           # Порт, на котором работает сервер PostgreSQL
)

In [11]:
cursor = connection.cursor()

In [12]:
delete_data = "DELETE FROM names" 

# Запускаем команду удаления данных
cursor.execute(delete_data)

In [13]:
cursor.execute("SELECT * FROM names")

In [14]:
cursor.fetchall()

[]

In [15]:
connection.rollback()

In [16]:
cursor.execute("SELECT * FROM names")
cursor.fetchall()

[(1, 'Мария', 252, 37750254, 2015, 'январь')]

Тестируем изоляцию транзакций

In [17]:
delete_data = "DELETE FROM names" 

# Запускаем команду удаления данных
cursor.execute(delete_data)

In [18]:
cursor.execute("SELECT * FROM names")
cursor.fetchall()

[]

In [19]:
# Оператор SQL для вставки данных в таблицу 
insert_several_rows = """
INSERT INTO names (id, name, number_of_persons, global_id, year, month)
VALUES (3, 'Анна', 190, 37750256, 2015, 'январь'),
       (4, 'Варвара', 190, 37750257, 2015, 'январь'),
       (5, 'Виктория', 186, 37750258, 2015, 'январь')
"""

In [20]:
# Запускаем команду вставки данных
cursor.execute(insert_several_rows)

In [21]:
cursor.execute("SELECT * FROM names")
cursor.fetchall()

[(3, 'Анна', 190, 37750256, 2015, 'январь'),
 (4, 'Варвара', 190, 37750257, 2015, 'январь'),
 (5, 'Виктория', 186, 37750258, 2015, 'январь')]

In [22]:
connection.commit()

## Работа с транзакциями через менеджер контекста

In [23]:
insert_data = """
INSERT INTO names (id, name, number_of_persons, global_id, year, month)
VALUES (2, 'Анастасия', 224, 37750255, 2015, 'январь')
"""

In [24]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute(insert_data)

После блока with транзакция фиксируется (если не было исключений).

Соединение при этом не закрывается.

In [25]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("SELECT * FROM names")
        for row in cursor.fetchall():
            print(row)


(3, 'Анна', 190, 37750256, 2015, 'январь')
(4, 'Варвара', 190, 37750257, 2015, 'январь')
(5, 'Виктория', 186, 37750258, 2015, 'январь')
(2, 'Анастасия', 224, 37750255, 2015, 'январь')


## Запросы с параметрами

In [26]:
insert_several_rows_parameters = """
INSERT INTO names (id, name, number_of_persons, global_id, year, month)
VALUES (%s, %s, %s, %s, %s, %s)
"""  

In [27]:
# Значения для параметров запроса
rows = [
    (483, 'Амелия', 8, 62367755, 2015, 'май'),
    (484, 'Камила', 8, 62367756, 2015, 'май')
]

In [28]:
with connection:
    with connection.cursor() as cursor:
        cursor.executemany(insert_several_rows_parameters, rows)
        cursor.execute("SELECT * FROM names")
        for row in cursor.fetchall():
            print(row)

(3, 'Анна', 190, 37750256, 2015, 'январь')
(4, 'Варвара', 190, 37750257, 2015, 'январь')
(5, 'Виктория', 186, 37750258, 2015, 'январь')
(2, 'Анастасия', 224, 37750255, 2015, 'январь')
(483, 'Амелия', 8, 62367755, 2015, 'май')
(484, 'Камила', 8, 62367756, 2015, 'май')


Команда с именованными параметрами

In [29]:
rows = [
    {'id': 483, 
     'name': 'Евангелина', 
     'number_of_persons': 8, 
     'global_id': 62367757, 
     'year': 2015, 
     'month': 'май'},
    {'id': 486, 
     'name': 'Альбина', 
     'number_of_persons': 8, 
     'global_id': 62367758, 
     'year': 2015, 
     'month': 'май'}
]

In [30]:
# Подготавливаем запрос с именованными параметрами
insert_several_rows_parameters = """
INSERT INTO names (id, name, number_of_persons, global_id, year, month)
VALUES (%(id)s, %(name)s, %(number_of_persons)s, %(global_id)s, %(year)s, %(month)s)
"""

In [31]:
with connection:
    with connection.cursor() as cursor:
        # Запускаем команду вставки нескольких элементов данных
        cursor.executemany(insert_several_rows_parameters, rows)
        cursor.execute("SELECT * FROM names")
        for row in cursor.fetchall():
            print(row)

(3, 'Анна', 190, 37750256, 2015, 'январь')
(4, 'Варвара', 190, 37750257, 2015, 'январь')
(5, 'Виктория', 186, 37750258, 2015, 'январь')
(2, 'Анастасия', 224, 37750255, 2015, 'январь')
(483, 'Амелия', 8, 62367755, 2015, 'май')
(484, 'Камила', 8, 62367756, 2015, 'май')
(483, 'Евангелина', 8, 62367757, 2015, 'май')
(486, 'Альбина', 8, 62367758, 2015, 'май')


## Расширение psycopg2.extras

In [32]:
from psycopg2.extras import execute_batch

In [33]:
insert_several_rows_parameters = """
INSERT INTO names (id, name, number_of_persons, global_id, year, month)
VALUES (%s, %s, %s, %s, %s, %s)
"""  

In [34]:
# Значения для параметров запроса
rows = [
    (483, 'Амелия', 8, 62367755, 2015, 'май'),
    (484, 'Камила', 8, 62367756, 2015, 'май')
]

In [35]:
with connection:
    with connection.cursor() as cursor:
        execute_batch(cursor, insert_several_rows_parameters, rows)
        cursor.execute("SELECT * FROM names")
        for row in cursor.fetchall():
            print(row)

(3, 'Анна', 190, 37750256, 2015, 'январь')
(4, 'Варвара', 190, 37750257, 2015, 'январь')
(5, 'Виктория', 186, 37750258, 2015, 'январь')
(2, 'Анастасия', 224, 37750255, 2015, 'январь')
(483, 'Амелия', 8, 62367755, 2015, 'май')
(484, 'Камила', 8, 62367756, 2015, 'май')
(483, 'Евангелина', 8, 62367757, 2015, 'май')
(486, 'Альбина', 8, 62367758, 2015, 'май')
(483, 'Амелия', 8, 62367755, 2015, 'май')
(484, 'Камила', 8, 62367756, 2015, 'май')


## Загружаем данные из JSON в PostgreSQL

In [36]:
import json

In [37]:
with open("names_m.json", "r") as read_file:
    data = json.load(read_file)

In [38]:
data

[{'ID': 1,
  'Name': 'Александр',
  'NumberOfPersons': 253,
  'global_id': 37750243,
  'Year': 2015,
  'Month': 'январь'},
 {'ID': 2,
  'Name': 'Михаил',
  'NumberOfPersons': 247,
  'global_id': 37750244,
  'Year': 2015,
  'Month': 'январь'},
 {'ID': 3,
  'Name': 'Иван',
  'NumberOfPersons': 226,
  'global_id': 37750245,
  'Year': 2015,
  'Month': 'январь'},
 {'ID': 4,
  'Name': 'Максим',
  'NumberOfPersons': 208,
  'global_id': 37750246,
  'Year': 2015,
  'Month': 'январь'},
 {'ID': 5,
  'Name': 'Артём',
  'NumberOfPersons': 190,
  'global_id': 37750247,
  'Year': 2015,
  'Month': 'январь'},
 {'ID': 6,
  'Name': 'Даниил',
  'NumberOfPersons': 171,
  'global_id': 62361364,
  'Year': 2015,
  'Month': 'январь'},
 {'ID': 7,
  'Name': 'Дмитрий',
  'NumberOfPersons': 164,
  'global_id': 62361365,
  'Year': 2015,
  'Month': 'январь'},
 {'ID': 8,
  'Name': 'Кирилл',
  'NumberOfPersons': 151,
  'global_id': 62361366,
  'Year': 2015,
  'Month': 'январь'},
 {'ID': 9,
  'Name': 'Андрей',
  'Numbe

In [39]:
len(data)

9901

In [42]:
# Оператор SQL для создания таблицы
create_names_table = """
CREATE TABLE IF NOT EXISTS names(
    id INTEGER,
    name TEXT,
    number_of_persons INTEGER,
    global_id BIGINT,
    year INTEGER,
    month TEXT 
)
"""

In [43]:
# Подготавливаем запрос с именованными параметрами
insert_several_rows_parameters = """
INSERT INTO names (id, name, number_of_persons, global_id, year, month)
VALUES (%(ID)s, %(Name)s, %(NumberOfPersons)s, %(global_id)s, %(Year)s, %(Month)s)
"""

In [44]:
with connection:
    with connection.cursor() as cursor:
        # Удаляем таблицу
        cursor.execute("DROP TABLE IF EXISTS names")
        # Запускаем команду создания таблицы
        cursor.execute(create_names_table)

In [45]:
with connection:
    with connection.cursor() as cursor:
        # Вставляем данные в таблицу
        execute_batch(cursor, insert_several_rows_parameters, data)

## Серверные курсоры

In [46]:
with connection:
    with connection.cursor(name='select_names') as cursor:
        # Количество строк данных, получаемых за один раз
        cursor.itersize = 10
        cursor.execute("SELECT * FROM names")
        for row in cursor:
            print(row)

(1, 'Александр', 253, 37750243, 2015, 'январь')
(2, 'Михаил', 247, 37750244, 2015, 'январь')
(3, 'Иван', 226, 37750245, 2015, 'январь')
(4, 'Максим', 208, 37750246, 2015, 'январь')
(5, 'Артём', 190, 37750247, 2015, 'январь')
(6, 'Даниил', 171, 62361364, 2015, 'январь')
(7, 'Дмитрий', 164, 62361365, 2015, 'январь')
(8, 'Кирилл', 151, 62361366, 2015, 'январь')
(9, 'Андрей', 130, 62361367, 2015, 'январь')
(10, 'Егор', 126, 62361368, 2015, 'январь')
(11, 'Илья', 122, 62361369, 2015, 'январь')
(12, 'Алексей', 108, 62361370, 2015, 'январь')
(13, 'Владимир', 107, 62361371, 2015, 'январь')
(14, 'Матвей', 102, 62361372, 2015, 'январь')
(15, 'Тимофей', 102, 62361373, 2015, 'январь')
(16, 'Ярослав', 91, 62361374, 2015, 'январь')
(17, 'Роман', 90, 62361375, 2015, 'январь')
(18, 'Фёдор', 87, 62361376, 2015, 'январь')
(19, 'Никита', 87, 62361377, 2015, 'январь')
(20, 'Марк', 85, 62361378, 2015, 'январь')
(21, 'Сергей', 84, 62361379, 2015, 'январь')
(22, 'Николай', 70, 62361380, 2015, 'январь')
(23, 